## Архитектуры свёрточных сетей

In [ ]:
import torch
from torch import nn
import torchvision as tv
from torchsummary import summary
import time

In [ ]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0, 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(device), y.to(device)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [ ]:
def train(net, train_iter, test_iter, trainer, num_epochs):
    net.to(device)
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        
        for i, (X, y) in enumerate(train_iter):
            X, y = X.to(device), y.to(device)
            trainer.zero_grad()
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]

            if i % 10 == 0:
              print(f"Step {i}. time since epoch: {time.time() -  start:.3f}. " 
                    f"Train acc: {train_acc_sum / n:.3f}. Train Loss: {train_l_sum / n:.3f}")
        test_acc = evaluate_accuracy(test_iter, net.to(device))
        print('-' * 20)
        print(f'epoch {epoch + 1}, loss {train_l_sum / n:.4f}, train acc {train_acc_sum / n:.3f}'
              f', test acc {test_acc:.3f}, time {time.time() - start:.1f} sec')

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
#torch.backends.cuda.max_split_size_mb = 10000

## DataSet

In [ ]:
BATCH_SIZE = 192
transoforms = tv.transforms.Compose([
    tv.transforms.Resize((224, 224)),
    tv.transforms.ToTensor()
])

train_dataset = tv.datasets.EMNIST('../data', split='balanced', train=True, download=True, transform=transoforms)
test_dataset = tv.datasets.EMNIST('../data', split='balanced', train=False, download=True, transform=transoforms)
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

## Предобученные архитектуры

In [ ]:
transoforms = tv.transforms.Compose([
    tv.transforms.Grayscale(3),
    tv.transforms.Resize((224, 224)),
    tv.transforms.ToTensor()
])

train_dataset = tv.datasets.MNIST('.', train=True, transform=transoforms, download=True)
test_dataset = tv.datasets.MNIST('.', train=False, transform=transoforms, download=True)
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

#### ResNet

In [ ]:
model = tv.models.resnet18(pretrained=True)

In [ ]:
model

In [ ]:
summary(model.to(device), input_size=(3, 224, 224))

In [ ]:
# Убираем требование градиента:
for param in model.parameters():
    param.requires_grad = False

In [ ]:
model.fc

In [ ]:
model.fc = nn.Linear(in_features=512, out_features=10)

In [ ]:
print("Params to learn:")
params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

In [ ]:
trainer = torch.optim.Adam(params_to_update, lr=0.001)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
train_loss_resnet18 = train(model, train_iter, test_iter, trainer, 1)

#### vgg16

In [ ]:
# VGG11
model_vgg16 = tv.models.vgg16(pretrained=True)


In [ ]:
model_vgg16

In [ ]:
# Убираем требование градиента:
for param in model_vgg16.parameters():
    param.requires_grad = False

In [ ]:
model_vgg16.classifier

In [ ]:
model_vgg16.classifier = nn.Sequential(
    nn.Linear(25088, 4096), 
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(4096, 10)
)

In [ ]:
# Распечатать заголовок секции 
print("Params to learn:")  

# Создать пустой список для хранения обучаемых параметров
params_to_update = []  

# Получить именованные параметры модели в виде списка tuple
for name, param in model_vgg16.named_parameters():
    
    # Проверить флаг requires_grad, который указывает нужно ли обучать этот параметр
    if param.requires_grad == True:

        # Добавить параметр в список обучаемых
        params_to_update.append(param)
        
        # Распечатать имя параметра с отступом
        print("\t",name)
        
# Теперь в params_to_update будут только обучаемые параметры модели

In [ ]:
trainer = torch.optim.Adam(params_to_update, lr=0.001)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
train_loss_vgg16 = train(model_vgg16, train_iter, test_iter, trainer, 1)

In [ ]:
torch.cuda.empty_cache()

### densenet161

In [ ]:
model = tv.models.densenet161(pretrained=True)
# Убираем требование градиента:
for param in model.parameters():
    param.requires_grad = False
model.classifier = nn.Linear(in_features=2208, out_features=10)
params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
trainer = torch.optim.Adam(params_to_update, lr=0.001)
train_loss_densenet161=train(model, train_iter, test_iter, trainer, 1)

### Inception v3 

In [ ]:
BATCH_SIZE = 192
transoforms = tv.transforms.Compose([
    tv.transforms.Grayscale(3),
    tv.transforms.Resize((299, 299)),
    tv.transforms.ToTensor()
])

train_dataset = tv.datasets.EMNIST('.', train=True, transform=transoforms, download=True, split='mnist')
test_dataset = tv.datasets.EMNIST('.', train=False, transform=transoforms, download=True, split='mnist')
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [ ]:
model = tv.models.inception_v3(pretrained=True)
model = model.to(device)


In [ ]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0, 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(device), y.to(device)
        y_hat = net(X)
        acc_sum += (y_hat.argmax(axis=1) == y).sum()
        n += y.shape[0]
        return 0
    return acc_sum.item() / n


def train(net, train_iter, test_iter, trainer, num_epochs):
    train_loss = []
    net.to(device)
    loss = nn.CrossEntropyLoss(reduction='sum')
    #net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        net.train()
        for i, (X, y) in enumerate(train_iter):
            X, y = X.to(device), y.to(device)
            trainer.zero_grad()
            y_hat = net(X)
            l = loss(y_hat[0], y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat[0].argmax(axis=1) == y).sum().item()
            n += y.shape[0]

            if i % 100 == 0:
              print(f"Step {i}. time since epoch: {time.time() -  start:.3f}. "
                    f"Train acc: {train_acc_sum / n:.3f}. Train Loss: {train_l_sum / n:.3f}")
        test_acc = evaluate_accuracy(test_iter, net.to(device))
        print('-' * 20)
        train_loss.append(train_l_sum / n)
        print(f'epoch {epoch + 1}, loss {train_l_sum / n:.4f}, train acc {train_acc_sum / n:.3f}'
              f', test acc {test_acc:.3f}, time {time.time() - start:.1f} sec')
    return train_loss

In [ ]:
torch.cuda.empty_cache()

In [ ]:

# Убираем требование градиента:
for param in model.parameters():
    param.requires_grad = False
model.fc = nn.Linear(in_features=2048, out_features=10)
params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)
trainer = torch.optim.Adam(params_to_update, lr=0.001)
train_loss_inception_v3=train(model, train_iter, test_iter, trainer, 1)

In [ ]:
import matplotlib.pyplot as plt

models = ['ResNet18', 'VGG16', 'DenseNet161', 'InceptionV3']
train_losses = [train_loss_resnet18, train_loss_vgg16, train_loss_densenet161, train_loss_inception_v3]

plt.figure(figsize=(10,5))
for i, loss in enumerate(train_losses):
  plt.plot(loss, label=models[i])
plt.xlabel('Epoch')  
plt.ylabel('Loss')
plt.legend()
plt.grid()
plt.title('Training Loss Comparison')
plt.show()

print("| Model | Loss |")
print("| - | - |")
for i in range(len(models)):
  print(f"| {models[i]} | {train_losses[i][-1]:.3f} |")